In [1]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import QuantileTransformer
import gc
from sklearn.metrics import roc_auc_score

In [3]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/moa-make-foldsmoa-make-folds/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/moa-make-foldsmoa-make-folds/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [4]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [5]:
#归一化、
from sklearn.preprocessing import StandardScaler,MinMaxScaler
std = StandardScaler()
train_features[CELLS] = std.fit_transform(train_features[CELLS])
test_features[CELLS] = std.transform(test_features[CELLS])

std = StandardScaler()
train_features[GENES] = std.fit_transform(train_features[GENES])
test_features[GENES] = std.transform(test_features[GENES])

In [6]:
#RankGauss

for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=150,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [7]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [8]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
target = train[train_targets_scored.columns[:207]]
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [9]:
train_non_scored = train_features.merge(train_targets_nonscored, on='sig_id')
train_non_scored = train_non_scored[train_non_scored['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
target_non_scored = train_non_scored[train_targets_nonscored.columns[:403]]
train_non_scored = train_non_scored.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [10]:
target_non_scored_cols = target_non_scored.drop('sig_id', axis=1).columns.values.tolist()
folds = train.copy()
folds_non_scored = train_non_scored.copy()

In [11]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

In [12]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

In [13]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [14]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.45)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.1)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def recalibrate_layer(self, layer):
        if(torch.isnan(layer.weight_v).sum() > 0):
            print ('recalibrate layer.weight_v')
            layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
            layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

        if(torch.isnan(layer.weight).sum() > 0):
            print ('recalibrate layer.weight')
            layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
            layer.weight += 1e-7
    
    def forward(self, x):
        x = self.batch_norm1(x)
        self.recalibrate_layer(self.dense1)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        self.recalibrate_layer(self.dense2)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        
        x = self.dense3(x)
        self.recalibrate_layer(self.dense3)
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [15]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['fold_42','sig_id','fold_0','fold_1','fold_2','fold_3']]
len(feature_cols)

877

In [16]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 200
BATCH_SIZE = 784
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 1e-5
NFOLDS = 7           
EARLY_STOPPING_STEPS = 30
EARLY_STOP = True

pre_EPOCHS = 70
pre_EARLY_STOPPING_STEPS = 10

num_features=len(feature_cols)
num_targets=len(target_cols)
num_non_scored_targets=len(target_non_scored_cols)
hidden_size=700   # 1500 1300 1400 1000 900 800 1700  20000

In [17]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)

    trn_idx = train[train[f'fold_{seed}'] != fold].index
    val_idx = train[train[f'fold_{seed}'] == fold].index
    
    train_df = train[train[f'fold_{seed}'] != fold].reset_index(drop=True)
    valid_df = train[train[f'fold_{seed}'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.00)
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    model.load_state_dict(torch.load(f"../input/moa-nn-train-no-pre/FOLD{fold}_{seed}.pth", map_location=torch.device('cpu'))) #
   # model.load_state_dict(torch.load(f"../input/moapytorch2model/FOLD{fold}_{seed}.pth")) #
    model.to(DEVICE)

    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
    oof[val_idx] = valid_preds
    print(valid_loss)

    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()
    return oof, predictions

In [18]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [19]:
# Averaging on multiple SEEDS

SEED = [42, 0] #<-- Update
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions

0.017039168160408735
0.0168904148042202
0.017645493149757385
0.01694003865122795
0.017236570827662945
0.01635933630168438
0.01650667702779174
0.016477983444929123
0.014100000588223338
0.017280043568462133
0.017373699322342874
0.01676855143159628
0.01765596978366375
0.0172235120087862


In [20]:
valid_results = train_targets_scored.drop(columns=target_cols+['fold_42','fold_0','fold_1','fold_2','fold_3']).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
valid_results = valid_results.add_prefix('pre_')
valid_results.rename(columns={'pre_sig_id':'sig_id'}, inplace=True)

oof = train_targets_scored.drop(columns=['fold_42','fold_0','fold_1','fold_2','fold_3'],axis=1)
oof = oof.merge(valid_results, on=['sig_id'], how='left')
oof.to_csv('moa_nn_oof.csv', index=False)

score = 0
for i in (target_cols):
    score_ = log_loss(oof[i], oof['pre_'+i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)
    

CV log_loss:  0.015564507825527934


In [21]:
auc_score = 0
for i in (target_cols):
    score_ = roc_auc_score(oof[i], oof['pre_'+i])
    score += score_ / target.shape[1]
    
print("CV AUC: ", score)

CV AUC:  0.6622805545183396


In [22]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)

In [23]:
sub.head(10)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000599,0.000752,0.002410,0.015671,0.019666,0.003828,0.002227,0.006960,0.000284,...,0.000942,0.001361,0.003675,0.001655,0.000951,0.000428,0.001274,0.001959,0.003688,0.001424
1,id_001897cda,0.000343,0.000599,0.001304,0.002373,0.002125,0.001508,0.004046,0.006565,0.002657,...,0.000566,0.000916,0.002705,0.000582,0.011588,0.000302,0.022500,0.000810,0.001007,0.002054
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000635,0.000697,0.001673,0.013418,0.016141,0.003909,0.003596,0.004501,0.000398,...,0.000602,0.001380,0.002357,0.016427,0.006405,0.000415,0.002479,0.001922,0.000701,0.001939
4,id_0027f1083,0.001403,0.001150,0.001496,0.016480,0.020921,0.004328,0.004023,0.002460,0.000390,...,0.000696,0.000617,0.003290,0.001747,0.001438,0.000488,0.002021,0.001644,0.000534,0.001842
5,id_0042c1364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,id_006fc47b8,0.000549,0.000550,0.001777,0.022360,0.022616,0.003585,0.004344,0.002179,0.000347,...,0.000825,0.001071,0.003145,0.004561,0.001463,0.000353,0.001162,0.001913,0.000386,0.001377
7,id_0071d65a2,0.000356,0.000505,0.000645,0.001540,0.003591,0.001485,0.000282,0.006066,0.000047,...,0.000159,0.000197,0.001577,0.001232,0.000544,0.000131,0.003023,0.000335,0.000467,0.000394
8,id_007a2159c,0.000531,0.000807,0.001375,0.006732,0.007536,0.002120,0.001688,0.010355,0.000261,...,0.000454,0.001368,0.002450,0.002787,0.001256,0.000463,0.001868,0.001190,0.000650,0.000568
9,id_009201382,0.000810,0.000909,0.001002,0.007002,0.008929,0.004416,0.004820,0.009433,0.000702,...,0.000505,0.001246,0.002153,0.002701,0.007566,0.000419,0.006189,0.001658,0.000583,0.002236
